In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

In [2]:
#train_data = np.genfromtxt('data/transformed_train.txt',delimiter = None)
#train_Y = np.genfromtxt('data/train_Y.txt',delimiter = None)
from imblearn.combine import SMOTETomek

#sc1 = StandardScaler()
#X = np.loadtxt("data/X_train.txt",delimiter=None)
#Y = np.loadtxt("data/Y_train.txt",delimiter=None)
train_data = np.loadtxt("data/transformed_train.txt",delimiter = None)
train_Y = np.loadtxt("data/train_Y.txt",delimiter = None)
#sc1 = StandardScaler()
#sc1.fit(train_data)
#train_data = sc1.transform(train_data)
#validation_data = np.loadtxt("data/reduced_validation.txt",delimiter = None)
#smotetomek = SMOTETomek(random_state = 0)
#sc1.fit(train_data)
#train_data = sc1.transform(train_data)
#train_data,train_Y = smotetomek.fit_sample(train_data,train_Y)

In [11]:
#Adaboost
alpha = np.zeros(shape=(84,1))
weights_models = np.ones(shape=(train_data.shape[0],))/train_data.shape[0]
validation_data = np.loadtxt('data/transformed_validation.txt',delimiter = None)
validation_Y = np.loadtxt('data/validation_Y.txt',delimiter = None)
#predictions_val = np.zeros(shape=(val_data.shape[0],val_data.shape[1]))
for i in range(0,84):
    model = DecisionTreeClassifier(max_depth = 1)
    model.fit(train_data,train_Y,np.atleast_1d(weights_models))
    joblib.dump(model,'data/AdaBoost/decision_model_'+str(i)+'.txt')
    predictions = model.predict(train_data)
    error = np.dot(weights_models,train_Y!=predictions)
    alpha[i] = 0.5*np.log((1-error)/error)
    weights_models *= np.exp(-alpha[i]*train_Y*predictions)
    weights_models /= weights_models.sum()

In [12]:
validation_predictions = np.zeros(shape=(validation_data.shape[0],))
for i in range(0,84):
#    for j in range(0,1):
    model = joblib.load('data/AdaBoost/decision_model_'+str(i)+'.txt')
    validation_predictions += model.predict(validation_data)
validation_predictions = np.sign(validation_predictions)
#model = LogisticRegression()
#model.fit(validation_predictions.reshape(-1,1),validation_Y)

In [142]:
#validation_predictions

In [6]:
np.count_nonzero(validation_Y==0)

12332

In [13]:
print(roc_auc_score(validation_Y,validation_predictions))
print(confusion_matrix(validation_Y,validation_predictions))

0.616972105092
[[8439 3893]
 [5554 6778]]


In [8]:
print(np.count_nonzero(validation_Y==1))

12332


In [14]:
test_data = np.loadtxt('data/transformed_test.txt',delimiter = None)
test_predictions = np.zeros(shape=(test_data.shape[0],))
for i in range(0,84):
#    for j in range(0,1):
    model = joblib.load('data/AdaBoost/decision_model_'+str(i)+'.txt')
    test_predictions += model.predict(test_data)
test_predictions = np.sign(test_predictions)
print(np.count_nonzero(test_predictions==1))

78332


In [148]:
#max(test_values)

In [160]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

adaboost = AdaBoostClassifier()
gbc = GradientBoostingClassifier()
adaboost.fit(train_data,train_Y)
gbc.fit(train_data,train_Y)
predictions = adaboost.predict(validation_data)
gbc_predictions = gbc.predict(validation_data)
print(roc_auc_score(validation_Y,predictions))
print(confusion_matrix(validation_Y,predictions))
print(roc_auc_score(validation_Y,gbc_predictions))
print(confusion_matrix(validation_Y,gbc_predictions))

0.640853065196
[[8761 3571]
 [5287 7045]]
0.684195588712
[[8972 3360]
 [4429 7903]]


In [162]:
predictions = gbc.predict(test_data)
print(np.count_nonzero(predictions==1))

74273


In [ ]:
training_data = np.loadtxt('data/reduced_train_svd.txt',delimiter = None)
training_Y = np.loadtxt('data/train_Y.txt',delimiter = None)
adaboost = AdaBoostClassifier()


In [149]:
Yte = np.vstack((np.arange(test_data.shape[0]), predictions.T)).T
np.savetxt('data/Y_submit.txt', Yte, '%d, %.2f', header='ID,Prob1', comments='', delimiter=',')